In [6]:
import spacy 
import torch
import json
from nlp_utils import *
import random

# Prep

In [7]:
with open ("data.json", 'r') as f:
    intents = json.load(f)["intents"]
    
data = torch.load("model.save")

model_state = data["model_state"]

input_size = data["input_size"]
hidden_size = data["hidden_size"]
num_classes = data["num_classes"]

tag_translator = data["tag_translator"]

device = torch.device("cuda")
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
model.load_state_dict(model_state)
model.eval()
nlp = spacy.load("en_core_web_lg")


# Bot

In [10]:
print("Type 'quit' to quit.")

user_input = ""

tag = ""


# while user_input != "quit":
while tag != "GoodBye":
    user_input = input()
    print("\tYou:",user_input)
    
    user_input_vector = torch.from_numpy(tokenize_to_vec(user_input, nlp))
    user_input_vector = user_input_vector.reshape(1, user_input_vector.shape[0])
    user_input_vector = user_input_vector.to(device)
    
    output = model(user_input_vector)
    _, prediction = torch.max(output, dim=1)

    probs = torch.softmax(output, dim=1)
    prob = probs[0][prediction.item()]


    if prob >= 0.75: 
        # get tag
        for key in tag_translator:
            if tag_translator[key] == prediction.item():
                tag = key
        # get response
        for intent in intents:
            if intent["intent"] == tag:
                print("\tChatBot:", random.choice(intent["responses"]))
                print()
                print("prob:", prob.item(), "| tag:", tag)
                print()
                print()
    else:
        print("prob:", prob.item())
        print("Chatbot: I don't understand.")
        print()
        print()
            
        
        
    

    

Type 'quit' to quit.
	You: hi
	ChatBot: Hi human, please tell me your GeniSys user

prob: 1.0 | tag: Greeting


	You: what is your name 
	ChatBot: Call me Geni

prob: 0.996103048324585 | tag: NameQuery


	You: you dont know this 
prob: 0.6570475101470947
Chatbot: I don't understand.


	You: see ya later 
	ChatBot: See you later

prob: 0.9999645948410034 | tag: GoodBye


